In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
file = "/content/sample_data/names.txt"
words = open(file, 'r').read().splitlines()

Mounted at /content/drive


In [2]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [16]:
import jax.numpy as jnp
import flax.linen as nn
from jax import random as jrand

In [8]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      # print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = jnp.array(X)
  Y = jnp.array(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

(182424, 3) (182424,)
(22836, 3) (22836,)
(22886, 3) (22886,)


In [12]:
Xtr.shape, Ytr.shape # dataset

((182424, 3), (182424,))

In [29]:
class MLP(nn.Module):                    # create a Flax Module dataclass
  out_dims: int

  @nn.compact
  def __call__(self, x):
    x = x.reshape((x.shape[0], -1))
    x = nn.Dense(128)(x)                 # create inline Flax Module submodules
    x = nn.relu(x)
    x = nn.Dense(self.out_dims)(x)       # shape inference
    return x

model = MLP(out_dims=10)                 # instantiate the MLP model

variables = model.init(jrand.PRNGKey(42), Xtr)   # initialize the weights
ypred = model.apply(variables, Xtr)            # make forward pass

In [30]:
print(ypred)

[[  0.           0.           0.         ...   0.           0.
    0.        ]
 [ -1.549385     1.1581401    3.9109955  ...   9.256715     0.36870533
   -7.1365685 ]
 [ -5.8675942    4.907771    -3.6533298  ...   6.919122     3.9543676
  -17.98331   ]
 ...
 [-13.749264     0.28428558  -3.8214917  ...   7.0870366   14.090424
  -11.266693  ]
 [-11.456205    -0.11970448   3.3698568  ...  12.974055     4.3092194
  -11.987412  ]
 [ -4.798683     5.6485534   -5.9289436  ...   3.5573502    7.177225
   -9.64392   ]]
